In [3]:
%load_ext autoreload
%autoreload 2
%env WANDB_SILENT=true

from pathlib import Path
from copy import deepcopy
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import quantstats as qs
import os
import logging
import sys

qs.extend_pandas()
plt.style.use("seaborn-whitegrid")
pd.set_option('display.max_rows', 200)

if "freqtrade" not in os.listdir():
    import nest_asyncio
    nest_asyncio.apply()
    os.chdir("..")
    
PATH_MOUNT = Path.cwd().parent / "mount"
print("Working Directory:", os.getcwd())

# Freqtrade Imports
from freqtrade.nbtools import remote_utils

env: WANDB_SILENT=true
Working Directory: C:\CS\Python\crypto-collection\freqtrade\superft


## Local Section

In [4]:
# Get local presets
local = (PATH_MOUNT / "presets").glob("*")
local = (it.name for it in list(local) if not it.name.startswith("."))
local_df = pd.DataFrame({"preset_name": local})
local_df

,preset_name
0,15m-next30m-10_06
1,15m-next30m-combine
2,bigz07
3,cluc-mad9
4,nfi


## Cloud Section

In [26]:
# Preset Backtest Results
df_download = remote_utils.cloud_get_presets_df(from_run_history=False)
df_download

,preset_name,backtest_date,leverage,direction,is_hedging,num_pairs,data_source,win_rate,avg_profit_winners_abs,avg_profit_losers_abs,...,max_drawdown_low,max_drawdown_high,csum_min,csum_max,profit_mean_abs,preset_type,fee,random_name,locks,pairlist
0,ma_crossover__backtest-2021-07-12_22-50-00,2021-07-12_22-50-00,1,long,False,1,binance,0.355623,0.175973,-0.162930,...,-14.728091,-0.048229,985.271909,999.951771,-0.042408,None,NaN,None,None,None
1,ma_crossover__backtest-2021-07-12_22-50-00,2021-07-12_22-50-00,1,long,False,1,binance,0.355623,0.175973,-0.162930,...,-14.728091,-0.048229,985.271909,999.951771,-0.042408,None,NaN,None,None,None
2,ma_crossover__backtest-2021-07-12_22-55-41,2021-07-12_22-55-41,1,long,False,1,binance,0.355623,0.175973,-0.162930,...,-14.728091,-0.048229,985.271909,999.951771,-0.042408,None,NaN,None,None,None
3,ma_crossover__backtest-2021-07-12_22-55-41,2021-07-12_22-55-41,1,long,False,1,binance,0.355623,0.175973,-0.162930,...,-14.728091,-0.048229,985.271909,999.951771,-0.042408,None,NaN,None,None,None
4,ma_crossover__dry__run-2021-07-12_23-13-05,2021-07-12_22-55-41,1,long,False,1,binance,0.355623,0.175973,-0.162930,...,-14.728091,-0.048229,985.271909,999.951771,-0.042408,None,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,DIY_MACDLongTerm__backtest-2021-08-20_21-15-07,2021-08-20 21:15:07,1,long,False,60,binance,0.401333,2.469692,-0.715104,...,58.462518,80.742041,99.768915,522.295562,0.563061,FilePreset,0.001,disgusted-shinchan-armor,[],None
302,DIY_MACDLongTerm__backtest-2021-08-20_21-15-33,2021-08-20 21:15:33,1,long,False,60,binance,0.370767,1.179550,-0.448950,...,348.763497,384.995555,95.252947,484.995555,0.154844,FilePreset,0.001,bipolar-yoshiko-table,[],None
303,DIY_MACDLongTerm__backtest-2021-08-20_21-15-50,2021-08-20 21:15:50,1,long,False,60,binance,0.401333,2.469692,-0.715104,...,58.462518,80.742041,99.768915,522.295562,0.563061,FilePreset,0.001,healthy-sasuke-mask,[],None
304,DIY_MACDLongTerm__backtest-2021-08-20_21-16-26,2021-08-20 21:16:26,1,long,False,60,binance,0.370767,1.179550,-0.448950,...,348.763497,384.995555,95.252947,484.995555,0.154844,FilePreset,0.001,sadistic-kazuya-laptop,[],None


In [33]:
# Get top 20 strategies by drawdown, but backtest_end is not older than 7 days. Drop duplicates by profit factor.
df = df_download.copy().drop_duplicates(subset=["profit_factor"])

df["profit_mean_pct"] = df["profit_mean"] * 100
df["backtest_start"] = pd.to_datetime(df["backtest_start"])
df["backtest_end"] = pd.to_datetime(df["backtest_end"])

columns = [
    "backtest_date", "preset_name", "timeframe", "profit_mean_pct", "profit_factor", 
    "expectancy_abs", "backtest_start" ,"backtest_end", "stake_amount", "max_open_trades", "holding_avg", 
    "strategy_name",
]

newer = pd.to_datetime(datetime.today() - timedelta(days=7))
last_month_btstart = pd.to_datetime(datetime.today() - timedelta(days=63))

df.loc[
    (df["profit_mean_pct"] > 0.25) & (df["backtest_end"] > newer) & (df["backtest_start"] > last_month_btstart), 
columns].sort_values(by=["profit_factor"], ascending=False).head(20)

,backtest_date,preset_name,timeframe,profit_mean_pct,profit_factor,expectancy_abs,backtest_start,backtest_end,stake_amount,max_open_trades,holding_avg,strategy_name
233,2021-08-19 20:05:56,SMAOffsetProtectOptV1__backtest-2021-08-19_20-...,5m,0.728675,5.794573,0.109411,2021-06-27 15:10:00,2021-08-17 14:00:00,15,6,0:56:00,SMAOffsetProtectOptV1
220,2021-08-19 19:50:54,BigZ04_TSL3__backtest-2021-08-19_19-50-54,5m,1.755044,5.497301,0.263520,2021-06-28 05:20:00,2021-08-17 14:00:00,15,6,8:29:00,BigZ04_TSL3
232,2021-08-19 20:05:31,SMAOffsetProtectOptV0__backtest-2021-08-19_20-...,5m,0.567497,2.965092,0.085210,2021-06-27 15:10:00,2021-08-17 14:00:00,15,6,1:19:00,SMAOffsetProtectOptV0
231,2021-08-19 20:04:47,SMAOffset__backtest-2021-08-19_20-04-47,5m,1.183992,2.431084,0.177776,2021-06-27 15:10:00,2021-08-17 14:00:00,15,6,6:45:00,SMAOffset
219,2021-08-19 19:49:44,AwesomeMacd__backtest-2021-08-19_19-49-44,1h,2.041753,2.066901,0.306569,2021-07-01 00:00:00,2021-08-17 12:00:00,15,6,"1 day, 11:10:00",AwesomeMacd
230,2021-08-19 20:03:06,Obelisk_Ichimoku_ZEMA_v1__backtest-2021-08-19_...,5m,0.837339,2.009617,0.101210,2021-06-29 06:20:00,2021-08-17 14:00:00,15,6,5:57:00,Obelisk_Ichimoku_ZEMA_v1
223,2021-08-19 19:58:14,CombinedBinHAndClucV5__backtest-2021-08-19_19-...,5m,0.450997,1.669355,0.067717,2021-06-27 16:50:00,2021-08-17 14:00:00,15,6,2:59:00,CombinedBinHAndClucV5
221,2021-08-19 19:56:29,CombinedBinHAndCluc__backtest-2021-08-19_19-56-29,5m,0.312835,1.535554,0.046972,2021-07-01 00:00:00,2021-08-17 14:00:00,15,6,2:17:00,CombinedBinHAndCluc
235,2021-08-19 20:06:52,SwingHighToSky__backtest-2021-08-19_20-06-52,15m,0.280145,1.514001,-1.236032,2021-07-01 00:00:00,2021-08-19 12:30:00,15,6,22:10:00,SwingHighToSky
234,2021-08-19 20:06:25,SMAOffsetV2__backtest-2021-08-19_20-06-25,5m,0.452526,1.499230,0.067947,2021-06-28 05:20:00,2021-08-17 14:00:00,15,6,2:52:00,SMAOffsetV2


In [34]:
# Long term strategies
df.loc[(df["profit_mean_pct"] > 0.25), columns].sort_values(by=["profit_mean_pct"], ascending=False).head(20)

,backtest_date,preset_name,timeframe,profit_mean_pct,profit_factor,expectancy_abs,backtest_start,backtest_end,stake_amount,max_open_trades,holding_avg,strategy_name
108,2021-07-23 23:31:36,nfi__backtest-2021-07-23_23-31-36,5m,9298.363377,3036.335814,1396.149261,2021-05-01 00:00:00,2021-07-17 15:50:00,15,6,10:26:00,NotebookStrategy
110,2021-07-23 23:53:35,nfi-down__backtest-2021-07-23_23-53-35,5m,9247.953611,2973.062888,1388.580235,2021-05-01 00:00:00,2021-07-17 15:50:00,15,6,"2 days, 6:08:00",NotebookStrategy
269,2021-08-20 15:33:44,MACDLongTerm__backtest-2021-08-20_15-33-44,3d,81.208232,36.337232,67.369199,2017-08-17 00:00:00,2021-07-08 00:00:00,-1,3,"51 days, 12:00:00",MACDLongTerm
250,2021-08-20 15:19:52,MACDLongTerm__backtest-2021-08-20_15-19-52,1d,17.930184,5.661347,5.384434,2018-04-17 00:00:00,2021-08-17 00:00:00,30,1,"12 days, 14:24:00",MACDLongTerm
252,2021-08-20 15:20:52,MACDLongTerm__backtest-2021-08-20_15-20-52,1d,17.930184,4.975515,35.990136,2018-04-17 00:00:00,2021-08-17 00:00:00,-1,1,"12 days, 14:24:00",MACDLongTerm
251,2021-08-20 15:20:37,MACDLongTerm__backtest-2021-08-20_15-20-37,1d,17.930184,4.975515,3.599014,2018-04-17 00:00:00,2021-08-17 00:00:00,-1,1,"12 days, 14:24:00",MACDLongTerm
298,2021-08-20 21:06:49,DIY_MACDLongTerm__backtest-2021-08-20_21-06-49,1d,17.196912,4.064634,2.582116,2017-08-17 00:00:00,2021-08-17 00:00:00,15,3,"11 days, 14:35:00",DIY_MACDLongTerm
268,2021-08-20 15:33:32,MACDLongTerm__backtest-2021-08-20_15-33-32,1d,15.360884,6.101224,26.284746,2017-08-17 00:00:00,2021-08-17 00:00:00,-1,3,"13 days, 5:51:00",MACDLongTerm
257,2021-08-20 15:24:33,MACDLongTerm__backtest-2021-08-20_15-24-33,1d,13.864072,3.703971,20.538102,2017-08-17 00:00:00,2021-08-17 00:00:00,-1,2,"12 days, 0:00:00",MACDLongTerm
249,2021-08-20 15:03:50,MACDLongTerm__backtest-2021-08-20_15-03-50,1d,13.864072,4.683061,4.163381,2017-08-17 00:00:00,2021-08-17 00:00:00,30,2,"12 days, 0:00:00",MACDLongTerm


In [32]:
# Most recent backtests
df.loc[(df["profit_mean_pct"] > 0.25), columns].sort_values(by=["backtest_date"], ascending=False).head(20)

,backtest_date,preset_name,timeframe,profit_mean_pct,profit_factor,expectancy_abs,backtest_start,backtest_end,stake_amount,max_open_trades,holding_avg
298,2021-08-20 21:06:49,DIY_MACDLongTerm__backtest-2021-08-20_21-06-49,1d,17.196912,4.064634,2.582116,2017-08-17 00:00:00,2021-08-17 00:00:00,15,3,"11 days, 14:35:00"
297,2021-08-20 21:06:30,DIY_MACDLongTerm__backtest-2021-08-20_21-06-30,12h,7.921821,3.029528,1.189462,2017-08-17 00:00:00,2021-08-18 12:00:00,15,3,"5 days, 14:58:00"
296,2021-08-20 21:05:45,DIY_MACDLongTerm__backtest-2021-08-20_21-05-45,6h,3.749988,2.315228,0.563061,2017-08-17 00:00:00,2021-08-19 00:00:00,15,3,"2 days, 14:46:00"
292,2021-08-20 21:03:33,DIY_MACDLongTerm__backtest-2021-08-20_21-03-33,2h,1.031264,1.548133,0.154844,2017-08-17 04:00:00,2021-08-19 08:00:00,15,3,19:41:00
291,2021-08-20 21:03:05,DIY_MACDLongTerm__backtest-2021-08-20_21-03-05,4h,1.496611,1.565836,0.224716,2017-08-17 04:00:00,2021-08-19 04:00:00,15,3,"1 day, 14:58:00"
290,2021-08-20 21:02:19,DIY_MACDLongTerm__backtest-2021-08-20_21-02-19,8h,4.745859,2.428338,0.712591,2017-08-17 00:00:00,2021-08-18 16:00:00,15,5,"3 days, 12:38:00"
288,2021-08-20 21:01:16,DIY_MACDLongTerm__backtest-2021-08-20_21-01-16,8h,5.806717,6.090637,3094.783635,2017-08-17 00:00:00,2021-08-18 16:00:00,-1,1,"3 days, 14:03:00"
287,2021-08-20 21:00:51,DIY_MACDLongTerm__backtest-2021-08-20_21-00-51,8h,5.860866,3.557324,892.266729,2017-08-17 00:00:00,2021-08-18 16:00:00,-1,2,"3 days, 15:47:00"
286,2021-08-20 21:00:38,DIY_MACDLongTerm__backtest-2021-08-20_21-00-38,8h,5.206228,2.374418,21.393279,2017-08-17 00:00:00,2021-08-18 16:00:00,-1,8,"3 days, 14:14:00"
285,2021-08-20 20:59:33,DIY_MACDLongTerm__backtest-2021-08-20_20-59-33,8h,4.745859,2.427406,38.487499,2017-08-17 00:00:00,2021-08-18 16:00:00,-1,5,"3 days, 12:38:00"


In [15]:
df_download.columns

Index(['preset_name', 'backtest_date', 'leverage', 'direction', 'is_hedging',
       'num_pairs', 'data_source', 'win_rate', 'avg_profit_winners_abs',
       'avg_profit_losers_abs', 'sum_profit_winners_abs',
       'sum_profit_losers_abs', 'profit_factor', 'profit_per_drawdown',
       'expectancy_abs', 'total_trades', 'total_volume', 'avg_stake_amount',
       'profit_mean', 'profit_median', 'profit_total', 'profit_total_abs',
       'backtest_start', 'backtest_start_ts', 'backtest_end',
       'backtest_end_ts', 'backtest_days', 'backtest_run_start_ts',
       'backtest_run_end_ts', 'trades_per_day', 'market_change',
       'stake_amount', 'stake_currency', 'stake_currency_decimals',
       'starting_balance', 'dry_run_wallet', 'final_balance',
       'rejected_signals', 'max_open_trades', 'max_open_trades_setting',
       'timeframe', 'timerange', 'enable_protections', 'strategy_name',
       'stoploss', 'trailing_stop', 'trailing_stop_positive',
       'trailing_stop_positive_offs

In [8]:
# Preset Run History
columns = ["preset_name", "profit_mean_pct", "profit_factor"]
df = remote_utils.cloud_get_presets_df(from_run_history=True)
df["profit_mean_pct"] = df["profit_mean"] * 100
df.loc[df["profit_mean_pct"] > 0.25, columns].sort_values(by=["profit_factor"], ascending=False).head(20)

,preset_name,profit_mean_pct,profit_factor
